# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
city_vac_df = pd.read_csv("../starter_code/CityData.csv")
city_vac_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Conakry,GN,2020-04-21 01:21:38,9.54,-13.68,80.83,74,37,3.58
1,1,Poum,NC,2020-04-21 01:21:38,-20.23,164.02,78.28,76,2,15.95
2,2,Avarua,CK,2020-04-21 01:21:38,-21.21,-159.78,82.40,74,1,17.22
3,3,Tiksi,RU,2020-04-21 01:21:39,71.69,128.87,23.11,95,95,3.00
4,4,Qaanaaq,GL,2020-04-21 01:21:39,77.48,-69.36,6.60,82,12,5.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
city_coord = city_vac_df[["Lat", "Lng"]]
city_humid = city_vac_df['Humidity']


fig = gmaps.figure()

# Create heat layer
heat_layer = (gmaps.heatmap_layer(locations=city_coord, weights=city_humid, dissipating=False, max_intensity=10, point_radius=1)) 
                                   
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
fav_cities_df = city_vac_df.copy()
fav_cities_df = fav_cities_df[fav_cities_df['Max Temp'] > 70]
fav_cities_df = fav_cities_df[fav_cities_df['Max Temp'] < 85]  

fav_cities_df = fav_cities_df[fav_cities_df['Cloudiness'] < 50]
fav_cities_df = fav_cities_df[fav_cities_df['Humidity'] < 50]

fav_cities_df = fav_cities_df.dropna()

fav_cities_df = fav_cities_df.reset_index()
fav_cities_df

,index,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,29,29,Adrar,MR,2020-04-21 01:21:43,20.50,-10.07,75.65,16,0,7.94
1,44,44,Taoudenni,ML,2020-04-21 01:21:03,22.68,-3.98,79.83,16,0,7.00
2,127,127,Benjamin Hill,MX,2020-04-21 01:21:59,30.17,-111.10,82.00,23,36,1.99
3,150,150,Bilma,NE,2020-04-21 01:22:02,18.69,12.92,83.44,12,13,5.30
4,197,197,Atar,MR,2020-04-21 01:22:10,20.52,-13.05,71.73,20,0,9.35
5,237,237,Portland,US,2020-04-21 01:20:36,45.52,-122.68,71.01,40,20,9.17
6,270,270,Carnarvon,AU,2020-04-21 01:22:22,-24.87,113.63,73.40,49,0,14.99
7,290,290,Puerto Penasco,MX,2020-04-21 01:22:25,31.32,-113.53,74.08,44,32,6.80
8,299,299,Cabo San Lucas,MX,2020-04-21 01:22:26,22.89,-109.91,79.00,41,5,10.29
9,300,300,Jalu,LY,2020-04-21 01:22:27,29.03,21.55,77.47,17,0,14.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = fav_cities_df
hotel_df["Hotel Name"] = ""
hotel_df

,index,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,29,29,Adrar,MR,2020-04-21 01:21:43,20.50,-10.07,75.65,16,0,7.94,
1,44,44,Taoudenni,ML,2020-04-21 01:21:03,22.68,-3.98,79.83,16,0,7.00,
2,127,127,Benjamin Hill,MX,2020-04-21 01:21:59,30.17,-111.10,82.00,23,36,1.99,
3,150,150,Bilma,NE,2020-04-21 01:22:02,18.69,12.92,83.44,12,13,5.30,
4,197,197,Atar,MR,2020-04-21 01:22:10,20.52,-13.05,71.73,20,0,9.35,
5,237,237,Portland,US,2020-04-21 01:20:36,45.52,-122.68,71.01,40,20,9.17,
6,270,270,Carnarvon,AU,2020-04-21 01:22:22,-24.87,113.63,73.40,49,0,14.99,
7,290,290,Puerto Penasco,MX,2020-04-21 01:22:25,31.32,-113.53,74.08,44,32,6.80,
8,299,299,Cabo San Lucas,MX,2020-04-21 01:22:26,22.89,-109.91,79.00,41,5,10.29,
9,300,300,Jalu,LY,2020-04-21 01:22:27,29.03,21.55,77.47,17,0,14.43,


In [6]:
# Use google API key to find hotels with 5000 meters of coordinates in cities
city_rec_ctr = 0
hotels_array = []

# Loop through each cities' coordinates to pull hotels in the area of each city
for cities in hotel_df['City']:
        city_coord = str(hotel_df['Lat'][city_rec_ctr])+' , '+str(hotel_df['Lng'][city_rec_ctr])
    
        #check in a radius of 5000 meters
        city_radius = 5000
        search_type = "lodging"
        city_rec_ctr +=1
     
        params = {"location": city_coord, "radius": city_radius, "type": search_type, "key": g_key}
    
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
        try:
            response = requests.get(base_url, params=params)
            hotel_list = response.json()
        
            hotels_array.append(hotel_list['results'][0]['name'])
        
        except IndexError:
            hotels_array.append(np.NaN)
            
# Add the names of the hotels to the dataframe
hotel_df['Hotel Name'] = hotels_array
hotel_df.head()

,index,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,29,29,Adrar,MR,2020-04-21 01:21:43,20.50,-10.07,75.65,16,0,7.94,NaN
1,44,44,Taoudenni,ML,2020-04-21 01:21:03,22.68,-3.98,79.83,16,0,7.00,NaN
2,127,127,Benjamin Hill,MX,2020-04-21 01:21:59,30.17,-111.10,82.00,23,36,1.99,Luis Guillermo García Medina
3,150,150,Bilma,NE,2020-04-21 01:22:02,18.69,12.92,83.44,12,13,5.30,Expeditions Ténére voyages
4,197,197,Atar,MR,2020-04-21 01:22:10,20.52,-13.05,71.73,20,0,9.35,Oumou Elghoura


In [7]:
# Dropping rows with NaN values or missing data
modDf = hotel_df.dropna(how='any')
 
print("Contents of the Modified Dataframe : ")
modDf.head()

Contents of the Modified Dataframe : 


,index,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
2,127,127,Benjamin Hill,MX,2020-04-21 01:21:59,30.17,-111.10,82.00,23,36,1.99,Luis Guillermo García Medina
3,150,150,Bilma,NE,2020-04-21 01:22:02,18.69,12.92,83.44,12,13,5.30,Expeditions Ténére voyages
4,197,197,Atar,MR,2020-04-21 01:22:10,20.52,-13.05,71.73,20,0,9.35,Oumou Elghoura
5,237,237,Portland,US,2020-04-21 01:20:36,45.52,-122.68,71.01,40,20,9.17,Portland Marriott City Center
6,270,270,Carnarvon,AU,2020-04-21 01:22:22,-24.87,113.63,73.40,49,0,14.99,Hospitality Carnarvon


###Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: Taoudenni.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: Benjamin Hill.
Closest hotel in Benjamin Hill is Luis Guillermo García Medina.
------------
Retrieving Results for Index 3: Bilma.
Closest hotel in Bilma is Expeditions Ténére voyages.
------------
Retrieving Results for Index 4: Atar.
Closest hotel in Atar is Oumou Elghoura.
------------
Retrieving Results for Index 5: Portland.
Closest hotel in Portland is Portland Marriott City Center.
------------
Retrieving Results for Index 6: Carnarvon.
Closest hotel in Carnarvon is Hospitality Carnarvon.
------------
Retrieving Results for Index 7: Puerto Penasco.
Closest hotel in Puerto Penasco is Playa Bonita Resort.
------------
Retrieving Results for Index 8: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Hotel Tesoro Los Cabos.
------------
Retrieving Results for Index 9: Jalu.
Clo

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))